---
## 1. Configuration & Setup

In [ ]:
# === IMPORTS ===
import os
import glob
import json
import warnings

# Data Processing
import numpy as np
import pandas as pd

# Visualization
import matplotlib.pyplot as plt
import seaborn as sns
from statsmodels.tsa.stattools import acf

# Deep Learning
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset

# Progress Bar
from tqdm.notebook import tqdm

# Suppress warnings
warnings.simplefilter("ignore")

# Plot settings
sns.set(style="ticks")
plt.rcParams['figure.figsize'] = (12, 6)
plt.rcParams['figure.dpi'] = 100

print(f"PyTorch version: {torch.__version__}")
print(f"CUDA available: {torch.cuda.is_available()}")

In [ ]:
# === CONFIGURATION ===
# All constants and hyperparameters in one place for easy modification

class Config:
    """Central configuration for the CAMELS prediction pipeline."""
    
    # === PATHS ===
    BASE_DIR = '../basin_dataset_public'
    FORCING_DIR = os.path.join(BASE_DIR, 'basin_mean_forcing', 'nldas')
    FLOW_DIR = os.path.join(BASE_DIR, 'usgs_streamflow')
    BAD_BASINS_FILE = os.path.join(BASE_DIR, 'basin_size_errors_10_percent.txt')
    RESULTS_DIR = 'results'
    
    # Static attribute files
    STATIC_FILES = [
        'camels_topo.txt',   # Elevation, Slope, Area
        'camels_soil.txt',   # Sand, Clay, Porosity
        'camels_clim.txt',   # Precip, Aridity, Snow Fraction
        'camels_vege.txt',   # Forest fraction, LAI
        'camels_geol.txt',   # Rock types, Permeability
    ]
    
    # === CONSTANTS ===
    CFS_TO_CMS = 0.0283168  # Cubic feet/s to Cubic meters/s
    
    # === EXPERIMENT SETTINGS ===
    USE_STATIC = True       # Include static catchment attributes
    NUM_BASINS = 5          # Number of basins to load (0 = all)
    
    # === MODEL HYPERPARAMETERS ===
    SEQ_LENGTH = 60         # Lookback window (days)
    PREDICT_HORIZON = 2     # Task 1: Predict t+k
    PREDICT_STEPS = 5       # Task 2: Predict sequence t+1...t+5
    
    HIDDEN_DIM = 64         # LSTM hidden units
    DROPOUT = 0.2           # Dropout rate
    LEARNING_RATE = 0.001   # Adam learning rate
    TEACHER_FORCING = 0.5   # Teacher forcing ratio for Seq2Seq
    
    # === TRAINING SETTINGS ===
    EPOCHS = 10
    BATCH_SIZE = 256
    USE_GPU = True
    
    # === DATA SPLIT (Hydrological Years) ===
    # Standard CAMELS split:
    TRAIN_START = '1980-10-01'
    TRAIN_END   = '1995-09-30'
    VAL_START   = '1995-10-01'
    VAL_END     = '2000-09-30'
    TEST_START  = '2000-10-01'
    TEST_END    = '2010-09-30'
    
    # === FEATURE SELECTION ===
    # Dynamic Inputs (Time-Series)
    DYNAMIC_FEATURES = [
        'PRCP', 'SRAD', 'Tmax', 'Tmin', 'Vp',   # Original Forcing
        'PRCP_roll3', 'PRCP_roll7',             # Rolling Stats
        'Q_lag1', 'Q_lag2', 'Q_lag3'            # Lag Features
    ]
    
    # Static Inputs (Basin Attributes)
    STATIC_FEATURES = [
        'area_gages2',  # Catchment Area (Log transformed)
        'elev_mean',    # Mean Elevation
        'slope_mean',   # Topography
        'sand_frac',    # Soil Type
        'clay_frac',    # Soil Type
        'frac_forest',  # Vegetation
        'lai_max',      # Leaf Area Index
        'p_mean',       # Long-term climate
        'aridity'       # Climate Index
    ]
    
    TARGET = 'Q_cms'  # Target variable (streamflow)

# Initialize config
cfg = Config()

# Set device
device = torch.device('cuda' if cfg.USE_GPU and torch.cuda.is_available() else 'cpu')
print(f"Using device: {device}")

In [ ]:
# === CHECK DATASET AVAILABILITY ===
dataset_exists = os.path.exists(cfg.BASE_DIR)

if dataset_exists:
    print(f"✓ Dataset found at: {os.path.abspath(cfg.BASE_DIR)}")
    # List contents
    contents = os.listdir(cfg.BASE_DIR)
    print(f"  Contents: {contents[:10]}{'...' if len(contents) > 10 else ''}")
else:
    print(f"✗ Dataset NOT found at: {os.path.abspath(cfg.BASE_DIR)}")
    print("\nPlease download the CAMELS dataset from:")
    print("  https://zenodo.org/records/15529996")
    print(f"\nAnd extract it to: {os.path.abspath(cfg.BASE_DIR)}")
    print("\nExpected structure:")
    print("  basin_dataset_public/")
    print("  ├── basin_mean_forcing/nldas/")
    print("  ├── usgs_streamflow/")
    print("  ├── camels_topo.txt")
    print("  ├── camels_soil.txt")
    print("  └── ...")

---
## 2. Data Loading

The `CamelsLoader` class handles all data I/O operations:
- Loading basin lists and filtering bad basins
- Loading dynamic time series (streamflow + forcing)
- Loading static catchment attributes

In [ ]:
# === DATA LOADER CLASS ===

class CamelsLoader:
    """Handles loading of CAMELS basin data."""
    
    def __init__(self, config):
        self.cfg = config
    
    def load_bad_basins(self):
        """Returns a list of basin IDs to exclude due to area errors."""
        if not os.path.exists(self.cfg.BAD_BASINS_FILE):
            return []
        
        bad_ids = []
        try:
            with open(self.cfg.BAD_BASINS_FILE, 'r') as f:
                next(f)  # Skip header
                for line in f:
                    parts = line.strip().split()
                    if len(parts) >= 2:
                        bad_ids.append(parts[1])
        except Exception as e:
            print(f"Failed to parse bad basins file: {e}")
        return bad_ids
    
    def get_basin_list(self):
        """Scans directories and returns DataFrame of valid basins."""
        bad_basins = self.load_bad_basins()
        
        search_path = os.path.join(self.cfg.FLOW_DIR, '**', '*_streamflow_qc.txt')
        files = glob.glob(search_path, recursive=True)
        
        basins = []
        for f in files:
            parts = f.split(os.sep)
            region = parts[-2]
            gauge_id = parts[-1].split('_')[0]
            
            if gauge_id not in bad_basins:
                basins.append({'gauge_id': gauge_id, 'region': region})
        
        return pd.DataFrame(basins)
    
    def load_dynamic_data(self, gauge_id, region):
        """
        Loads Streamflow + Forcing data for a single basin.
        Returns cleaned DataFrame with standard column names.
        """
        # 1. Load Streamflow
        flow_path = os.path.join(self.cfg.FLOW_DIR, region, f'{gauge_id}_streamflow_qc.txt')
        try:
            df_flow = pd.read_csv(flow_path, delim_whitespace=True, header=None,
                                  names=['gauge_id', 'Year', 'Month', 'Day', 'Q_cfs', 'QC'])
        except:
            return None
        
        df_flow['Date'] = pd.to_datetime(df_flow[['Year', 'Month', 'Day']])
        df_flow.set_index('Date', inplace=True)
        # Convert to CMS and mark missing values
        df_flow['Q_cms'] = df_flow['Q_cfs'].replace(-999, np.nan) * self.cfg.CFS_TO_CMS
        
        # 2. Load Forcing (NLDAS)
        forcing_path = os.path.join(self.cfg.FORCING_DIR, region, 
                                    f'{gauge_id}_lump_nldas_forcing_leap.txt')
        if not os.path.exists(forcing_path):
            return None
        
        try:
            df_force = pd.read_csv(forcing_path, delim_whitespace=True, skiprows=3)
        except:
            return None
        
        # Normalize column names (handle variations)
        col_map = {
            'Mnth': 'Month', 'month': 'Month', 'mo': 'Month',
            'year': 'Year', 'yr': 'Year',
            'day': 'Day', 'dy': 'Day',
            'prcp(mm/day)': 'PRCP', 'srad(w/m2)': 'SRAD',
            'tmax(c)': 'Tmax', 'tmin(c)': 'Tmin', 'vp(pa)': 'Vp'
        }
        
        new_cols = {}
        for c in df_force.columns:
            clean = c.strip()
            if clean.lower() in col_map:
                new_cols[c] = col_map[clean.lower()]
            elif clean in col_map:
                new_cols[c] = col_map[clean]
        
        df_force.rename(columns=new_cols, inplace=True)
        
        try:
            df_force['Date'] = pd.to_datetime(df_force[['Year', 'Month', 'Day']])
            df_force.set_index('Date', inplace=True)
        except KeyError:
            return None
        
        # 3. Merge flow and forcing
        base_forcing = ['PRCP', 'SRAD', 'Tmax', 'Tmin', 'Vp']
        cols_to_use = [c for c in base_forcing if c in df_force.columns]
        df_merged = df_flow[['Q_cms']].join(df_force[cols_to_use], how='inner')
        
        return df_merged
    
    def load_static_attributes(self, basins_list=None):
        """
        Loads all attribute files, merges them, and filters for requested features.
        """
        dfs = []
        for filename in self.cfg.STATIC_FILES:
            path = os.path.join(self.cfg.BASE_DIR, filename)
            if os.path.exists(path):
                try:
                    df = pd.read_csv(path, sep=';')
                    df.columns = [c.strip() for c in df.columns]
                    if 'gauge_id' in df.columns:
                        df['gauge_id'] = df['gauge_id'].astype(str).str.zfill(8)
                        df.set_index('gauge_id', inplace=True)
                        dfs.append(df)
                except:
                    pass
        
        if not dfs:
            return None
        
        # Merge all static files
        df_static = pd.concat(dfs, axis=1)
        df_static = df_static.loc[:, ~df_static.columns.duplicated()]
        
        # Filter for configured features
        available_feats = [f for f in self.cfg.STATIC_FEATURES if f in df_static.columns]
        df_final = df_static[available_feats]
        
        if basins_list is not None:
            df_final = df_final.reindex(basins_list)
        
        return df_final

# Initialize loader
loader = CamelsLoader(cfg)
print("CamelsLoader initialized.")

In [ ]:
# === LOAD BASIN LIST ===
if dataset_exists:
    df_basins = loader.get_basin_list()
    
    print(f"Total valid basins found: {len(df_basins)}")
    print(f"\nRegion distribution:")
    print(df_basins['region'].value_counts().head(10))
    
    # Preview
    display(df_basins.head())
else:
    print("Dataset not available. Please download first.")

In [ ]:
# === LOAD SAMPLE BASIN DATA ===
if dataset_exists and len(df_basins) > 0:
    # Load one basin for inspection
    sample_row = df_basins.iloc[0]
    sample_id = sample_row['gauge_id']
    sample_region = sample_row['region']
    
    print(f"Loading sample basin: {sample_id} (Region: {sample_region})")
    df_sample = loader.load_dynamic_data(sample_id, sample_region)
    
    if df_sample is not None:
        print(f"\nShape: {df_sample.shape}")
        print(f"Date range: {df_sample.index.min()} to {df_sample.index.max()}")
        print(f"\nColumns: {df_sample.columns.tolist()}")
        print(f"\nMissing values:")
        print(df_sample.isna().sum())
        print(f"\n")
        display(df_sample.head(10))
    else:
        print("Failed to load sample basin.")
else:
    print("Dataset not available.")

In [ ]:
# === LOAD STATIC ATTRIBUTES ===
if dataset_exists:
    df_static = loader.load_static_attributes()
    
    if df_static is not None:
        print(f"Static attributes shape: {df_static.shape}")
        print(f"\nFeatures loaded: {df_static.columns.tolist()}")
        print(f"\n")
        display(df_static.head())
    else:
        print("No static attribute files found.")
else:
    print("Dataset not available.")

---
## 3. Exploratory Data Analysis (EDA)

Visualizations to understand:
- Single basin time series characteristics
- Flow duration curves
- Seasonality patterns
- Autocorrelation and cross-correlation (lag analysis)
- Global statistics across all basins
- Static feature distributions

In [ ]:
# === 3.1 SINGLE BASIN ANALYSIS ===
if dataset_exists and df_sample is not None:
    fig, axes = plt.subplots(1, 3, figsize=(18, 5))
    
    # 1. Hydrograph (2 year subset)
    try:
        subset = df_sample['1995':'1996']
        if subset.empty:
            raise ValueError("Empty slice")
        title_text = "Hydrograph (1995-1996)"
    except:
        subset = df_sample.iloc[:730]
        title_text = "Hydrograph (First 2 Years)"
    
    axes[0].plot(subset.index, subset['Q_cms'], label='Flow', color='steelblue')
    axes[0].set_title(title_text)
    axes[0].set_ylabel("Flow (m³/s)")
    axes[0].set_xlabel("Date")
    
    # 2. Flow Duration Curve (FDC)
    sorted_flow = np.sort(df_sample['Q_cms'].dropna())[::-1]
    exceedance = np.arange(1, len(sorted_flow)+1) / len(sorted_flow) * 100
    axes[1].semilogy(exceedance, sorted_flow, color='teal')
    axes[1].set_title("Flow Duration Curve (Log Scale)")
    axes[1].set_xlabel("Exceedance Probability (%)")
    axes[1].set_ylabel("Flow (m³/s)")
    axes[1].grid(True, which="both", ls="-", alpha=0.3)
    
    # 3. Seasonality Check (Tmax vs Day of Year)
    if 'Tmax' in df_sample.columns:
        axes[2].scatter(df_sample.index.dayofyear, df_sample['Tmax'], 
                       s=1, alpha=0.3, c='coral')
        axes[2].set_title("Seasonality Check: Tmax vs Day of Year")
        axes[2].set_xlabel("Day of Year")
        axes[2].set_ylabel("Temperature (°C)")
    
    plt.tight_layout()
    plt.show()
else:
    print("Dataset not available for EDA.")

In [ ]:
# === 3.2 LAG ANALYSIS FOR FORECASTING ===
if dataset_exists and df_sample is not None:
    clean_df = df_sample.dropna()
    lags = 10
    
    # Calculate correlations
    auto_corr = acf(clean_df['Q_cms'], nlags=lags)
    cross_corr = [clean_df['Q_cms'].corr(clean_df['PRCP'].shift(i)) for i in range(lags)]
    
    fig, ax = plt.subplots(1, 2, figsize=(14, 4))
    
    # Autocorrelation (Memory)
    ax[0].bar(range(len(auto_corr)), auto_corr, color='teal')
    ax[0].set_title("Target Autocorrelation (How dependent is Q on past Q?)")
    ax[0].set_xlabel("Lag (Days)")
    ax[0].set_ylabel("Correlation")
    ax[0].axhline(y=0, color='gray', linestyle='--', alpha=0.5)
    
    # Cross Correlation (Response Time)
    ax[1].stem(range(lags), cross_corr)
    ax[1].set_title("Precip-Flow Cross Correlation (Rain-to-River Lag)")
    ax[1].set_xlabel("Lag (Days)")
    ax[1].set_ylabel("Correlation")
    
    plt.tight_layout()
    plt.show()
    
    print("\\n📊 Key Insights:")
    print(f"  • Lag-1 autocorrelation: {auto_corr[1]:.3f} (High = Strong memory)")
    print(f"  • Peak PRCP-Q correlation at lag: {np.argmax(cross_corr)} days")
else:
    print("Dataset not available for lag analysis.")

In [ ]:
# === 3.3 GLOBAL STATISTICS (ALL BASINS) ===
# This cell processes multiple basins - may take a few minutes

if dataset_exists:
    global_stats = []
    basins_to_process = df_basins.head(min(100, len(df_basins)))  # Limit for speed
    
    print(f"Processing {len(basins_to_process)} basins for global statistics...")
    
    for idx, row in tqdm(basins_to_process.iterrows(), total=len(basins_to_process)):
        df = loader.load_dynamic_data(row['gauge_id'], row['region'])
        
        if df is None or len(df) < 365:
            continue
        
        total_days = len(df)
        missing_q = df['Q_cms'].isna().sum()
        neg_flows = (df['Q_cms'] < 0).sum()
        
        mean_q = df['Q_cms'].mean()
        std_q = df['Q_cms'].std()
        cv_q = std_q / mean_q if mean_q > 0 else 0
        
        lag1_corr = df['Q_cms'].corr(df['Q_cms'].shift(1))
        
        global_stats.append({
            'gauge_id': row['gauge_id'],
            'region': row['region'],
            'missing_pct': (missing_q / total_days) * 100,
            'negative_q_count': neg_flows,
            'mean_flow_cms': mean_q,
            'cv_flow': cv_q,
            'lag1_autocorr': lag1_corr
        })
    
    df_global = pd.DataFrame(global_stats)
    
    print("\\n[Global Data Quality Summary]")
    display(df_global.describe().T[['mean', 'min', '50%', 'max']])
else:
    print("Dataset not available.")

In [ ]:
# === 3.4 GLOBAL DISTRIBUTION PLOTS ===
if dataset_exists and len(global_stats) > 0:
    fig, axes = plt.subplots(2, 2, figsize=(15, 10))
    
    # 1. Missing Data Histogram
    sns.histplot(df_global['missing_pct'], bins=30, ax=axes[0,0], color='salmon')
    axes[0,0].set_title("Distribution of Missing Data (%)")
    axes[0,0].set_xlabel("% Missing Days")
    
    # 2. Basin Scale (Mean Flow - Log)
    sns.histplot(df_global['mean_flow_cms'], bins=50, log_scale=True, ax=axes[0,1], color='steelblue')
    axes[0,1].set_title("Distribution of Mean Streamflow (Log Scale)")
    axes[0,1].set_xlabel("Mean Flow (m³/s)")
    
    # 3. System Memory (Lag-1 Autocorrelation)
    sns.histplot(df_global['lag1_autocorr'], bins=30, ax=axes[1,0], color='green')
    axes[1,0].set_title("Distribution of Lag-1 Autocorrelation (System Memory)")
    axes[1,0].set_xlabel("Correlation r(t, t-1)")
    
    # 4. Flow Variability (CV)
    sns.histplot(df_global['cv_flow'], bins=30, ax=axes[1,1], color='purple')
    axes[1,1].set_title("Distribution of Flow Coefficient of Variation")
    axes[1,1].set_xlabel("CV (std/mean)")
    
    plt.tight_layout()
    plt.show()
else:
    print("Global statistics not available.")

In [ ]:
# === 3.5 STATIC FEATURES ANALYSIS ===
if dataset_exists and df_static is not None:
    # Key features to visualize
    key_features = ['area_gages2', 'elev_mean', 'aridity', 'sand_frac', 'frac_forest', 'p_mean']
    plot_feats = [f for f in key_features if f in df_static.columns]
    
    if plot_feats:
        fig, axes = plt.subplots(2, 3, figsize=(15, 10))
        axes = axes.flatten()
        
        for i, feat in enumerate(plot_feats):
            sns.histplot(df_static[feat].dropna(), bins=30, ax=axes[i], color='teal')
            axes[i].set_title(f"Distribution of {feat}")
            axes[i].set_xlabel(feat)
        
        plt.suptitle("Key Static Feature Distributions", fontsize=14, y=1.02)
        plt.tight_layout()
        plt.show()
else:
    print("Static features not available.")

---
## 4. Feature Engineering

Create derived features to improve model performance:
- **Rolling Statistics**: 3-day and 7-day precipitation averages (soil moisture proxy)
- **Lag Features**: Previous streamflow values (autoregressive input)

In [ ]:
# === FEATURE ENGINEER CLASS ===

class FeatureEngineer:
    """Creates derived features for the prediction model."""
    
    def __init__(self, config):
        self.cfg = config
    
    def add_rolling_features(self, df):
        """
        Calculates rolling statistics for Precipitation.
        Purpose: Represents accumulated soil moisture / wetness.
        """
        if 'PRCP' in df.columns:
            # 3-Day Rolling Mean (Short-term wetness)
            df['PRCP_roll3'] = df['PRCP'].rolling(window=3, min_periods=1).mean()
            # 7-Day Rolling Mean (Medium-term saturation)
            df['PRCP_roll7'] = df['PRCP'].rolling(window=7, min_periods=1).mean()
        return df
    
    def add_lag_features(self, df):
        """
        Adds explicit lag features for Streamflow.
        Purpose: Give model explicit access to recent flow values.
        """
        target = self.cfg.TARGET
        
        if target in df.columns:
            df['Q_lag1'] = df[target].shift(1)
            df['Q_lag2'] = df[target].shift(2)
            df['Q_lag3'] = df[target].shift(3)
            
            # Backfill NaNs created by shifting
            for col in ['Q_lag1', 'Q_lag2', 'Q_lag3']:
                df[col] = df[col].bfill()
        
        return df
    
    def transform(self, df):
        """
        Run all engineering steps.
        Should be run AFTER cleaning, BEFORE normalization.
        """
        df = self.add_rolling_features(df)
        df = self.add_lag_features(df)
        return df

# Initialize feature engineer
engineer = FeatureEngineer(cfg)
print("FeatureEngineer initialized.")

In [ ]:
# === TEST FEATURE ENGINEERING ===
if dataset_exists and df_sample is not None:
    # Apply feature engineering to sample
    df_engineered = engineer.transform(df_sample.copy())
    
    print("New features added:")
    new_cols = ['PRCP_roll3', 'PRCP_roll7', 'Q_lag1', 'Q_lag2', 'Q_lag3']
    print(f"  {[c for c in new_cols if c in df_engineered.columns]}")
    
    print(f"\\nShape before: {df_sample.shape}")
    print(f"Shape after:  {df_engineered.shape}")
    
    display(df_engineered.head(10))
else:
    print("Dataset not available.")

---
## 5. Preprocessing

The `CamelsPreprocessor` handles:
- **Physical Outlier Cleaning**: Remove impossible values (negative rain, extreme temps)
- **Missing Data Handling**: Interpolation and fill strategies
- **Date Features**: Cyclical sin/cos encoding for day-of-year
- **Normalization**: Global dynamic normalization + basin-specific target normalization
- **Sequence Creation**: Sliding window sequences for LSTM input

In [ ]:
# === PREPROCESSOR CLASS ===

class CamelsPreprocessor:
    """Handles data cleaning, normalization, and sequence creation."""
    
    def __init__(self, config):
        self.cfg = config
        self.scalers = {}        # Global normalization stats
        self.basin_scalers = {}  # Per-basin target stats
        
        # Physical Constraints
        self.PHYSICAL_LIMITS = {
            'PRCP': {'min': 0.0, 'max': None},
            'Q_cms': {'min': 0.0, 'max': None},
            'Tmax': {'min': -60.0, 'max': 60.0},
            'Tmin': {'min': -60.0, 'max': 60.0}
        }
        self.MAX_INTERPOLATE_GAP = 5
    
    def add_date_features(self, df):
        """Add cyclical day-of-year features."""
        day_of_year = df.index.dayofyear
        df['sin_doy'] = np.sin(2 * np.pi * day_of_year / 365.0)
        df['cos_doy'] = np.cos(2 * np.pi * day_of_year / 365.0)
        return df
    
    def clean_physical_outliers(self, df):
        """Remove physically impossible values."""
        # Negative Rain/Flow -> 0
        for col in ['PRCP', self.cfg.TARGET]:
            if col in df.columns:
                mask = df[col] < 0
                if mask.any():
                    df.loc[mask, col] = 0.0
        
        # Unrealistic Temp -> NaN
        for col in ['Tmax', 'Tmin']:
            if col in df.columns:
                limits = self.PHYSICAL_LIMITS[col]
                mask = (df[col] < limits['min']) | (df[col] > limits['max'])
                if mask.any():
                    df.loc[mask, col] = np.nan
        return df
    
    def handle_missing_data(self, df):
        """Interpolate and fill missing values."""
        cols_to_fix = [self.cfg.TARGET] + self.cfg.DYNAMIC_FEATURES
        cols_to_fix = [c for c in cols_to_fix if c in df.columns]
        
        for col in cols_to_fix:
            df[col] = df[col].interpolate(method='linear', 
                                          limit=self.MAX_INTERPOLATE_GAP, 
                                          limit_direction='both')
            df[col] = df[col].ffill().bfill()
        return df
    
    def fit(self, dynamic_data_dict, static_df=None):
        """Compute normalization statistics from training data."""
        print("Computing global statistics...")
        
        # 1. Dynamic Stats (Global)
        dyn_vals = []
        for gid, df in dynamic_data_dict.items():
            train_slice = df.loc[self.cfg.TRAIN_START:self.cfg.TRAIN_END]
            if not train_slice.empty:
                available_cols = [c for c in self.cfg.DYNAMIC_FEATURES if c in train_slice.columns]
                valid_rows = train_slice[available_cols].dropna()
                if not valid_rows.empty:
                    dyn_vals.append(valid_rows.values)
        
        if dyn_vals:
            all_dyn = np.vstack(dyn_vals)
            self.scalers['dynamic_mean'] = np.mean(all_dyn, axis=0)
            self.scalers['dynamic_std'] = np.std(all_dyn, axis=0) + 1e-6
        else:
            self.scalers['dynamic_mean'] = 0
            self.scalers['dynamic_std'] = 1
        
        # 2. Static Stats (if provided)
        if static_df is not None:
            static_copy = static_df.copy()
            if 'area_gages2' in static_copy.columns:
                static_copy['area_gages2'] = np.log10(np.maximum(static_copy['area_gages2'], 1e-3))
            self.scalers['static_mean'] = static_copy.mean().values
            self.scalers['static_std'] = static_copy.std().values + 1e-6
        else:
            print("-> Skipping Static Stats (not provided)")
        
        # 3. Basin-Specific Target Stats
        print("Computing basin-specific target statistics...")
        for gid, df in dynamic_data_dict.items():
            train_slice = df.loc[self.cfg.TRAIN_START:self.cfg.TRAIN_END]
            clean_target = train_slice[self.cfg.TARGET].dropna()
            
            if not clean_target.empty:
                self.basin_scalers[gid] = {
                    'mean': clean_target.mean(),
                    'std': clean_target.std() + 1e-6
                }
            else:
                self.basin_scalers[gid] = {'mean': 0, 'std': 1}
    
    def transform(self, df_dynamic, df_static=None, gauge_id=None):
        """
        Normalize data and prepare for sequence creation.
        Returns: (data_matrix, static_norm_vector)
        """
        # 1. Normalize Dynamic Features
        dyn_cols = [c for c in self.cfg.DYNAMIC_FEATURES if c in df_dynamic.columns]
        X_dyn = df_dynamic[dyn_cols].values
        X_dyn_norm = (X_dyn - self.scalers['dynamic_mean']) / self.scalers['dynamic_std']
        
        # 2. Normalize Target (Basin-Specific)
        target = df_dynamic[self.cfg.TARGET].values
        b_stats = self.basin_scalers.get(gauge_id, {'mean': 0, 'std': 1})
        y_norm = (target - b_stats['mean']) / b_stats['std']
        
        # 3. Normalize Static (if available)
        X_stat_norm = None
        if df_static is not None and gauge_id in df_static.index:
            static_vals = df_static.loc[gauge_id].values.copy()
            if 'area_gages2' in df_static.columns:
                area_idx = df_static.columns.get_loc('area_gages2')
                static_vals[area_idx] = np.log10(np.maximum(static_vals[area_idx], 1e-3))
            X_stat_norm = (static_vals - self.scalers['static_mean']) / self.scalers['static_std']
        
        # 4. Date Features
        date_feats = df_dynamic[['sin_doy', 'cos_doy']].values
        
        # Combine: [Dynamic_Norm, Date, Target_Norm]
        data_matrix = np.column_stack([X_dyn_norm, date_feats, y_norm])
        
        return data_matrix, X_stat_norm
    
    def create_sequences(self, data_matrix, static_vec=None, mode='task1'):
        """
        Creates sliding window sequences for LSTM.
        mode='task1': Single-step prediction (t+2)
        mode='task2': Multi-step sequence prediction (t+1..t+5)
        """
        X_seq, y_seq = [], []
        seq_len = self.cfg.SEQ_LENGTH
        total_samples = len(data_matrix)
        
        use_static = (static_vec is not None)
        if use_static:
            static_repeated = np.tile(static_vec, (seq_len, 1))
        
        if mode == 'task1':
            horizon = self.cfg.PREDICT_HORIZON
            for t in range(seq_len, total_samples - horizon + 1):
                window_data = data_matrix[t-seq_len:t, :]
                target_val = data_matrix[t + horizon - 1, -1]
                
                if np.isnan(window_data).any() or np.isnan(target_val):
                    continue
                
                if use_static:
                    full_X = np.hstack([window_data, static_repeated])
                else:
                    full_X = window_data
                
                X_seq.append(full_X)
                y_seq.append(target_val)
        
        elif mode == 'task2':
            steps = self.cfg.PREDICT_STEPS
            for t in range(seq_len, total_samples - steps + 1):
                window_data = data_matrix[t-seq_len:t, :]
                target_seq = data_matrix[t:t+steps, -1]
                
                if np.isnan(window_data).any() or np.isnan(target_seq).any():
                    continue
                
                if use_static:
                    full_X = np.hstack([window_data, static_repeated])
                else:
                    full_X = window_data
                
                X_seq.append(full_X)
                y_seq.append(target_seq)
        
        return np.array(X_seq), np.array(y_seq)

# Initialize preprocessor
preprocessor = CamelsPreprocessor(cfg)
print("CamelsPreprocessor initialized.")

---
## 6. Model Definitions

### Model Architectures

**Task 1 - LSTM (Single-Step Prediction)**
```
Input (Batch, Seq_Len, Features) 
  → LSTM(hidden=64) 
  → Last Hidden State 
  → Dropout(0.2) 
  → Linear(1) 
  → Output (flow at t+2)
```

**Task 2 - Encoder-Decoder with Cross-Attention (Multi-Step)**
```
Encoder: Past sequence → LSTM → Hidden States
Decoder: For each step t in [1..5]:
  Input: [prev_flow, future_forcing[t], static]
  → LSTMCell 
  → CrossAttention(encoder_outputs) 
  → Linear(1) 
  → prediction[t]
```

In [ ]:
# === MODEL: LSTM (Task 1 - Single Step) ===

class LSTM(nn.Module):
    """Standard LSTM for Single-Step Prediction (t+2)."""
    
    def __init__(self, input_dim, hidden_dim, dropout=0.2):
        super(LSTM, self).__init__()
        
        self.lstm = nn.LSTM(
            input_size=input_dim,
            hidden_size=hidden_dim,
            num_layers=1,
            batch_first=True,
            dropout=0
        )
        
        self.dropout = nn.Dropout(dropout)
        self.head = nn.Linear(hidden_dim, 1)
    
    def forward(self, x):
        # x: (Batch, Seq_Len, Features)
        out, (h_n, c_n) = self.lstm(x)
        
        # Take last hidden state
        last_hidden = out[:, -1, :]  # (Batch, Hidden)
        
        out = self.dropout(last_hidden)
        prediction = self.head(out)  # (Batch, 1)
        
        return prediction

print("LSTM model class defined.")

In [ ]:
# === MODEL: Cross-Attention Mechanism ===

class CrossAttention(nn.Module):
    """Bahdanau-style attention for Seq2Seq decoder."""
    
    def __init__(self, hidden_dim):
        super(CrossAttention, self).__init__()
        self.attn = nn.Linear(hidden_dim * 2, hidden_dim)
        self.v = nn.Linear(hidden_dim, 1, bias=False)
    
    def forward(self, hidden, encoder_outputs):
        # hidden: (Batch, Hidden) - current decoder state
        # encoder_outputs: (Batch, Seq_Len, Hidden)
        
        seq_len = encoder_outputs.size(1)
        
        # Repeat decoder hidden for all encoder timesteps
        hidden_expanded = hidden.unsqueeze(1).repeat(1, seq_len, 1)
        
        # Calculate attention energy
        combined = torch.cat((hidden_expanded, encoder_outputs), dim=2)
        energy = torch.tanh(self.attn(combined))
        attention = self.v(energy).squeeze(2)  # (Batch, Seq_Len)
        
        # Softmax weights
        weights = F.softmax(attention, dim=1)
        
        # Context vector (weighted sum)
        context = torch.bmm(weights.unsqueeze(1), encoder_outputs)
        
        return context.squeeze(1), weights

print("CrossAttention module defined.")

In [ ]:
# === MODEL: LSTM_Seq2Seq (Task 2 - Multi-Step) ===

class LSTM_Seq2Seq(nn.Module):
    """Encoder-Decoder with Cross-Attention for sequence prediction."""
    
    def __init__(self, input_dim, hidden_dim, future_forcing_dim, static_dim, output_steps=5):
        super(LSTM_Seq2Seq, self).__init__()
        self.output_steps = output_steps
        self.static_dim = static_dim
        
        # Encoder
        self.encoder = nn.LSTM(input_dim, hidden_dim, batch_first=True)
        
        # Decoder components
        decoder_input_dim = 1 + future_forcing_dim + static_dim
        self.decoder_cell = nn.LSTMCell(decoder_input_dim, hidden_dim)
        self.attention = CrossAttention(hidden_dim)
        
        # Output projection
        self.fc_out = nn.Linear(hidden_dim * 2, 1)
    
    def forward(self, x_past, x_future_forcing, static_features, 
                target_seq=None, teacher_forcing_ratio=0.5):
        """
        Args:
            x_past: (Batch, Past_Seq, Input_Dim)
            x_future_forcing: (Batch, 5, Future_Forcing_Dim)
            static_features: (Batch, Static_Dim)
            target_seq: (Batch, 5) - Ground truth for teacher forcing
            teacher_forcing_ratio: Probability of using ground truth
        
        Returns:
            outputs: (Batch, 5) - Predicted flow sequence
        """
        batch_size = x_past.size(0)
        
        # 1. Encode past sequence
        encoder_outputs, (hidden, cell) = self.encoder(x_past)
        hidden = hidden.squeeze(0)
        cell = cell.squeeze(0)
        
        # Initialize outputs
        outputs = torch.zeros(batch_size, self.output_steps).to(x_past.device)
        
        # First decoder input: last observed flow
        decoder_input_flow = x_past[:, -1, -1].unsqueeze(1)
        
        # 2. Decode loop
        for t in range(self.output_steps):
            # Prepare decoder input
            current_forcing = x_future_forcing[:, t, :]
            inputs_list = [decoder_input_flow, current_forcing]
            
            if self.static_dim > 0 and static_features is not None:
                inputs_list.append(static_features)
            
            dec_input = torch.cat(inputs_list, dim=1)
            
            # Run decoder cell
            hidden, cell = self.decoder_cell(dec_input, (hidden, cell))
            
            # Cross attention
            context, _ = self.attention(hidden, encoder_outputs)
            
            # Predict
            combined = torch.cat((hidden, context), dim=1)
            prediction = self.fc_out(combined)
            
            outputs[:, t] = prediction.squeeze(1)
            
            # Teacher forcing
            if target_seq is not None and torch.rand(1).item() < teacher_forcing_ratio:
                decoder_input_flow = target_seq[:, t].unsqueeze(1)
            else:
                decoder_input_flow = prediction
        
        return outputs

print("LSTM_Seq2Seq model class defined.")

---
## 7. Training & Evaluation

### Evaluation Metric: Nash-Sutcliffe Efficiency (NSE)

$$NSE = 1 - \frac{\sum(Q_{obs} - Q_{sim})^2}{\sum(Q_{obs} - \bar{Q}_{obs})^2}$$

- **NSE = 1**: Perfect prediction
- **NSE = 0**: Model is as good as predicting the mean
- **NSE < 0**: Model is worse than the mean

In [ ]:
# === UTILITY FUNCTIONS ===

def calc_nse(obs, sim):
    """Calculate Nash-Sutcliffe Efficiency."""
    denominator = np.sum((obs - np.mean(obs)) ** 2) + 1e-6
    numerator = np.sum((sim - obs) ** 2)
    return 1 - (numerator / denominator)

def save_results(results, params, output_dir):
    """Save metrics and parameters to JSON."""
    if not os.path.exists(output_dir):
        os.makedirs(output_dir)
    
    with open(os.path.join(output_dir, 'metrics.json'), 'w') as f:
        json.dump(results, f, indent=4)
    
    with open(os.path.join(output_dir, 'params.json'), 'w') as f:
        params_clean = {k: str(v) for k, v in params.items()}
        json.dump(params_clean, f, indent=4)
    
    print(f"Results saved to {output_dir}")

print("Utility functions defined.")

In [ ]:
# === DATA PREPARATION PIPELINE ===

def prepare_data(config, loader, engineer, preprocessor, num_basins=5):
    """
    Full data preparation pipeline:
    1. Load basin list
    2. Load dynamic data for each basin
    3. Apply cleaning, feature engineering, date features
    4. Fit preprocessor (compute normalization stats)
    """
    print("--- 1. Loading Data ---")
    df_basins = loader.get_basin_list()
    
    if df_basins.empty:
        raise ValueError("No basins found or all filtered out.")
    
    total_found = len(df_basins)
    if num_basins > 0:
        print(f"Found {total_found} basins. Loading first {num_basins} for experiment...")
        df_basins = df_basins.head(num_basins)
    else:
        print(f"Loading all {total_found} basins...")
    
    basin_ids = df_basins['gauge_id'].tolist()
    df_static = loader.load_static_attributes(basin_ids) if config.USE_STATIC else None
    
    dynamic_data = {}
    print("Loading and preprocessing dynamic data...")
    for _, row in tqdm(df_basins.iterrows(), total=len(df_basins)):
        gid = row['gauge_id']
        region = row['region']
        
        df = loader.load_dynamic_data(gid, region)
        if df is not None:
            df = preprocessor.clean_physical_outliers(df)
            df = preprocessor.handle_missing_data(df)
            df = engineer.transform(df)
            df = preprocessor.add_date_features(df)
            dynamic_data[gid] = df
    
    print(f"Successfully loaded {len(dynamic_data)} basins.")
    
    print("\\n--- 2. Fitting Preprocessor ---")
    preprocessor.fit(dynamic_data, df_static)
    
    return dynamic_data, df_static, basin_ids

print("Data preparation function defined.")

In [ ]:
# === DATASET GENERATORS ===

def get_task1_dataset(config, dynamic_data, df_static, preprocessor, basin_ids, split='train'):
    """Generate dataset for Task 1 (single-step prediction)."""
    X_list, y_list = [], []
    
    if split == 'train':
        start, end = config.TRAIN_START, config.TRAIN_END
    elif split == 'val':
        start, end = config.VAL_START, config.VAL_END
    else:
        start, end = config.TEST_START, config.TEST_END
    
    for gid in basin_ids:
        if gid not in dynamic_data:
            continue
        df = dynamic_data[gid].loc[start:end]
        if df.empty:
            continue
        
        data_matrix, static_vec = preprocessor.transform(df, df_static, gid)
        X, y = preprocessor.create_sequences(
            data_matrix, 
            static_vec if config.USE_STATIC else None, 
            mode='task1'
        )
        
        if len(X) > 0:
            X_list.append(X)
            y_list.append(y)
    
    if not X_list:
        return None, None
    return np.concatenate(X_list), np.concatenate(y_list)


def get_task2_dataset(config, dynamic_data, df_static, preprocessor, basin_ids, split='train'):
    """Generate dataset for Task 2 (multi-step sequence prediction)."""
    X_past_list, X_future_list, Static_list, Y_list = [], [], [], []
    
    if split == 'train':
        start, end = config.TRAIN_START, config.TRAIN_END
    elif split == 'val':
        start, end = config.VAL_START, config.VAL_END
    else:
        start, end = config.TEST_START, config.TEST_END
    
    # Indices for forcing features (first 5 dynamic features + date features)
    forcing_indices = [0, 1, 2, 3, 4]  # PRCP, SRAD, Tmax, Tmin, Vp
    n_dyn = len(config.DYNAMIC_FEATURES)
    future_feat_indices = forcing_indices + [n_dyn, n_dyn + 1]  # + sin_doy, cos_doy
    
    for gid in basin_ids:
        if gid not in dynamic_data:
            continue
        df = dynamic_data[gid].loc[start:end]
        if df.empty:
            continue
        
        data_matrix, static_vec = preprocessor.transform(df, df_static, gid)
        
        seq_len = config.SEQ_LENGTH
        steps = config.PREDICT_STEPS
        total = len(data_matrix)
        
        static_repeated = np.tile(static_vec, (seq_len, 1)) if (config.USE_STATIC and static_vec is not None) else None
        
        c_xp, c_xf, c_st, c_y = [], [], [], []
        
        for t in range(seq_len, total - steps + 1):
            past_window = data_matrix[t-seq_len:t, :]
            future_window = data_matrix[t:t+steps, :][:, future_feat_indices]
            target_seq = data_matrix[t:t+steps, -1]
            
            if np.isnan(past_window).any() or np.isnan(future_window).any() or np.isnan(target_seq).any():
                continue
            
            if static_repeated is not None:
                full_past = np.hstack([past_window, static_repeated])
            else:
                full_past = past_window
            
            c_xp.append(full_past)
            c_xf.append(future_window)
            c_st.append(static_vec if static_vec is not None else np.zeros(1))
            c_y.append(target_seq)
        
        if len(c_xp) > 0:
            X_past_list.append(np.array(c_xp))
            X_future_list.append(np.array(c_xf))
            Static_list.append(np.array(c_st))
            Y_list.append(np.array(c_y))
    
    if not X_past_list:
        return None, None, None, None
    
    return (np.concatenate(X_past_list), np.concatenate(X_future_list),
            np.concatenate(Static_list), np.concatenate(Y_list))

print("Dataset generator functions defined.")

In [ ]:
# === TASK 1: TRAINING FUNCTION ===

def run_task1(config, device, dynamic_data, df_static, preprocessor, basin_ids):
    """Train and evaluate Task 1 (Single-Step Prediction)."""
    print("\\n" + "="*50)
    print(" TASK 1: Single Step Prediction (t+2)")
    print("="*50)
    
    # 1. Prepare datasets
    print("\\nPreparing datasets...")
    X_train, y_train = get_task1_dataset(config, dynamic_data, df_static, preprocessor, basin_ids, 'train')
    X_val, y_val = get_task1_dataset(config, dynamic_data, df_static, preprocessor, basin_ids, 'val')
    
    if X_train is None:
        print("Not enough data for Task 1.")
        return None
    
    print(f"Train samples: {len(X_train)}, Val samples: {len(X_val)}")
    
    train_ds = TensorDataset(torch.Tensor(X_train), torch.Tensor(y_train))
    val_ds = TensorDataset(torch.Tensor(X_val), torch.Tensor(y_val))
    train_loader = DataLoader(train_ds, batch_size=config.BATCH_SIZE, shuffle=True)
    val_loader = DataLoader(val_ds, batch_size=config.BATCH_SIZE, shuffle=False)
    
    # 2. Initialize model
    input_dim = X_train.shape[2]
    model = LSTM(input_dim=input_dim, hidden_dim=config.HIDDEN_DIM, dropout=config.DROPOUT).to(device)
    optimizer = optim.Adam(model.parameters(), lr=config.LEARNING_RATE)
    criterion = nn.MSELoss()
    
    print(f"\\nModel input dim: {input_dim}")
    print(f"Total parameters: {sum(p.numel() for p in model.parameters()):,}")
    
    # 3. Training loop
    best_loss = float('inf')
    save_path = os.path.join(config.RESULTS_DIR, 'task1', 'best_model.pth')
    os.makedirs(os.path.dirname(save_path), exist_ok=True)
    
    train_losses, val_losses = [], []
    
    print(f"\\nTraining for {config.EPOCHS} epochs...")
    for epoch in range(config.EPOCHS):
        # Train
        model.train()
        train_loss = 0
        for X_b, y_b in train_loader:
            X_b, y_b = X_b.to(device), y_b.to(device)
            optimizer.zero_grad()
            loss = criterion(model(X_b).squeeze(), y_b)
            loss.backward()
            optimizer.step()
            train_loss += loss.item()
        
        # Validate
        model.eval()
        val_loss = 0
        with torch.no_grad():
            for X_b, y_b in val_loader:
                X_b, y_b = X_b.to(device), y_b.to(device)
                val_loss += criterion(model(X_b).squeeze(), y_b).item()
        
        train_loss /= len(train_loader)
        val_loss /= len(val_loader)
        train_losses.append(train_loss)
        val_losses.append(val_loss)
        
        print(f"Epoch {epoch+1:02d}: Train Loss {train_loss:.4f} | Val Loss {val_loss:.4f}", end="")
        
        if val_loss < best_loss:
            best_loss = val_loss
            torch.save(model.state_dict(), save_path)
            print(" ✓ Best")
        else:
            print()
    
    # 4. Evaluate on test set
    print("\\nEvaluating on Test Set...")
    X_test, y_test = get_task1_dataset(config, dynamic_data, df_static, preprocessor, basin_ids, 'test')
    
    if X_test is not None:
        model.load_state_dict(torch.load(save_path, weights_only=True))
        model.eval()
        with torch.no_grad():
            preds = model(torch.Tensor(X_test).to(device)).cpu().numpy().squeeze()
        
        nse = calc_nse(y_test, preds)
        mse = float(np.mean((y_test - preds) ** 2))
        
        print(f"\\n📊 Task 1 Results:")
        print(f"   Test NSE: {nse:.4f}")
        print(f"   Test MSE: {mse:.4f}")
        
        # Save results
        save_results({'NSE': nse, 'Test_MSE': mse}, 
                    {'epochs': config.EPOCHS, 'batch_size': config.BATCH_SIZE}, 
                    os.path.join(config.RESULTS_DIR, 'task1'))
        
        return {
            'model': model,
            'train_losses': train_losses,
            'val_losses': val_losses,
            'test_nse': nse,
            'predictions': preds,
            'targets': y_test
        }
    
    return None

print("Task 1 training function defined.")

In [ ]:
# === TASK 2: TRAINING FUNCTION ===

def run_task2(config, device, dynamic_data, df_static, preprocessor, basin_ids):
    """Train and evaluate Task 2 (Multi-Step Sequence Prediction)."""
    print("\\n" + "="*50)
    print(" TASK 2: Multi-Step Sequence (t+1..t+5)")
    print("="*50)
    
    # 1. Prepare datasets
    print("\\nPreparing datasets...")
    tr_data = get_task2_dataset(config, dynamic_data, df_static, preprocessor, basin_ids, 'train')
    val_data = get_task2_dataset(config, dynamic_data, df_static, preprocessor, basin_ids, 'val')
    
    if tr_data[0] is None:
        print("Not enough data for Task 2.")
        return None
    
    print(f"Train samples: {len(tr_data[0])}, Val samples: {len(val_data[0])}")
    
    train_ds = TensorDataset(*[torch.Tensor(x) for x in tr_data])
    val_ds = TensorDataset(*[torch.Tensor(x) for x in val_data])
    train_loader = DataLoader(train_ds, batch_size=config.BATCH_SIZE, shuffle=True)
    val_loader = DataLoader(val_ds, batch_size=config.BATCH_SIZE, shuffle=False)
    
    # 2. Initialize model
    input_dim = tr_data[0].shape[2]
    future_dim = tr_data[1].shape[2]
    static_dim = tr_data[2].shape[1]
    
    model = LSTM_Seq2Seq(
        input_dim, config.HIDDEN_DIM, future_dim, static_dim, config.PREDICT_STEPS
    ).to(device)
    
    optimizer = optim.Adam(model.parameters(), lr=config.LEARNING_RATE)
    criterion = nn.MSELoss()
    
    print(f"\\nModel dims - Input: {input_dim}, Future: {future_dim}, Static: {static_dim}")
    print(f"Total parameters: {sum(p.numel() for p in model.parameters()):,}")
    
    # 3. Training loop
    best_loss = float('inf')
    save_path = os.path.join(config.RESULTS_DIR, 'task2', 'best_model.pth')
    os.makedirs(os.path.dirname(save_path), exist_ok=True)
    
    train_losses, val_losses = [], []
    
    print(f"\\nTraining for {config.EPOCHS} epochs...")
    for epoch in range(config.EPOCHS):
        # Train
        model.train()
        train_loss = 0
        for xp, xf, st, y in train_loader:
            xp, xf, st, y = xp.to(device), xf.to(device), st.to(device), y.to(device)
            optimizer.zero_grad()
            preds = model(xp, xf, st, target_seq=y, teacher_forcing_ratio=config.TEACHER_FORCING)
            loss = criterion(preds, y)
            loss.backward()
            optimizer.step()
            train_loss += loss.item()
        
        # Validate
        model.eval()
        val_loss = 0
        with torch.no_grad():
            for xp, xf, st, y in val_loader:
                xp, xf, st, y = xp.to(device), xf.to(device), st.to(device), y.to(device)
                preds = model(xp, xf, st, target_seq=None, teacher_forcing_ratio=0)
                val_loss += criterion(preds, y).item()
        
        train_loss /= len(train_loader)
        val_loss /= len(val_loader)
        train_losses.append(train_loss)
        val_losses.append(val_loss)
        
        print(f"Epoch {epoch+1:02d}: Train Loss {train_loss:.4f} | Val Loss {val_loss:.4f}", end="")
        
        if val_loss < best_loss:
            best_loss = val_loss
            torch.save(model.state_dict(), save_path)
            print(" ✓ Best")
        else:
            print()
    
    # 4. Evaluate on test set
    print("\\nEvaluating on Test Set...")
    te_data = get_task2_dataset(config, dynamic_data, df_static, preprocessor, basin_ids, 'test')
    
    if te_data[0] is not None:
        test_ds = TensorDataset(*[torch.Tensor(x) for x in te_data])
        test_loader = DataLoader(test_ds, batch_size=config.BATCH_SIZE, shuffle=False)
        
        model.load_state_dict(torch.load(save_path, weights_only=True))
        model.eval()
        
        all_preds, all_targets = [], []
        with torch.no_grad():
            for xp, xf, st, y in test_loader:
                xp, xf, st = xp.to(device), xf.to(device), st.to(device)
                out = model(xp, xf, st, target_seq=None, teacher_forcing_ratio=0)
                all_preds.append(out.cpu().numpy())
                all_targets.append(y.numpy())
        
        preds_flat = np.concatenate(all_preds).flatten()
        targets_flat = np.concatenate(all_targets).flatten()
        nse = calc_nse(targets_flat, preds_flat)
        
        print(f"\\n📊 Task 2 Results:")
        print(f"   Test NSE: {nse:.4f}")
        
        save_results({'NSE': nse}, 
                    {'epochs': config.EPOCHS, 'batch_size': config.BATCH_SIZE},
                    os.path.join(config.RESULTS_DIR, 'task2'))
        
        return {
            'model': model,
            'train_losses': train_losses,
            'val_losses': val_losses,
            'test_nse': nse,
            'predictions': np.concatenate(all_preds),
            'targets': np.concatenate(all_targets)
        }
    
    return None

print("Task 2 training function defined.")

In [ ]:
# === RUN FULL PIPELINE ===
# This cell executes the complete training pipeline

if dataset_exists:
    # Prepare data
    dynamic_data, df_static_train, basin_ids = prepare_data(
        cfg, loader, engineer, preprocessor, 
        num_basins=cfg.NUM_BASINS
    )
    
    # Train Task 1
    task1_results = run_task1(cfg, device, dynamic_data, df_static_train, preprocessor, basin_ids)
    
    # Train Task 2
    task2_results = run_task2(cfg, device, dynamic_data, df_static_train, preprocessor, basin_ids)
else:
    print("Dataset not available. Please download the CAMELS dataset first.")

---
## 8. Results & Analysis

Visualize training progress and model performance.

In [ ]:
# === 8.1 TRAINING CURVES ===

def plot_training_curves(results, task_name):
    """Plot training and validation loss curves."""
    if results is None:
        print(f"No results available for {task_name}")
        return
    
    fig, ax = plt.subplots(1, 1, figsize=(10, 5))
    
    epochs = range(1, len(results['train_losses']) + 1)
    ax.plot(epochs, results['train_losses'], 'b-', label='Train Loss', linewidth=2)
    ax.plot(epochs, results['val_losses'], 'r-', label='Val Loss', linewidth=2)
    
    ax.set_xlabel('Epoch')
    ax.set_ylabel('MSE Loss')
    ax.set_title(f'{task_name} - Training Progress (Test NSE: {results["test_nse"]:.4f})')
    ax.legend()
    ax.grid(True, alpha=0.3)
    
    plt.tight_layout()
    plt.show()

# Plot if results exist
if 'task1_results' in dir() and task1_results is not None:
    plot_training_curves(task1_results, "Task 1: Single-Step (t+2)")

if 'task2_results' in dir() and task2_results is not None:
    plot_training_curves(task2_results, "Task 2: Multi-Step (t+1..t+5)")

In [ ]:
# === 8.2 PREDICTED VS OBSERVED ===

def plot_predictions(results, task_name, n_samples=500):
    """Plot predicted vs observed scatter plot."""
    if results is None:
        print(f"No results available for {task_name}")
        return
    
    preds = results['predictions'].flatten()[:n_samples]
    targets = results['targets'].flatten()[:n_samples]
    
    fig, axes = plt.subplots(1, 2, figsize=(14, 5))
    
    # Scatter plot
    axes[0].scatter(targets, preds, alpha=0.3, s=10, c='steelblue')
    
    # 1:1 line
    min_val = min(targets.min(), preds.min())
    max_val = max(targets.max(), preds.max())
    axes[0].plot([min_val, max_val], [min_val, max_val], 'r--', linewidth=2, label='1:1 Line')
    
    axes[0].set_xlabel('Observed (Normalized)')
    axes[0].set_ylabel('Predicted (Normalized)')
    axes[0].set_title(f'{task_name} - Predicted vs Observed')
    axes[0].legend()
    axes[0].grid(True, alpha=0.3)
    
    # Residual histogram
    residuals = preds - targets
    axes[1].hist(residuals, bins=50, color='coral', edgecolor='black', alpha=0.7)
    axes[1].axvline(x=0, color='black', linestyle='--', linewidth=2)
    axes[1].set_xlabel('Residual (Pred - Obs)')
    axes[1].set_ylabel('Frequency')
    axes[1].set_title(f'{task_name} - Residual Distribution')
    axes[1].grid(True, alpha=0.3)
    
    plt.tight_layout()
    plt.show()
    
    # Statistics
    print(f"\\n📊 Residual Statistics:")
    print(f"   Mean: {np.mean(residuals):.4f}")
    print(f"   Std:  {np.std(residuals):.4f}")
    print(f"   Min:  {np.min(residuals):.4f}")
    print(f"   Max:  {np.max(residuals):.4f}")

# Plot if results exist
if 'task1_results' in dir() and task1_results is not None:
    plot_predictions(task1_results, "Task 1")

if 'task2_results' in dir() and task2_results is not None:
    plot_predictions(task2_results, "Task 2")

In [ ]:
# === 8.3 TIME SERIES VISUALIZATION ===

def plot_time_series(results, task_name, n_points=200):
    """Plot a segment of predicted vs observed time series."""
    if results is None:
        print(f"No results available for {task_name}")
        return
    
    preds = results['predictions'].flatten()[:n_points]
    targets = results['targets'].flatten()[:n_points]
    
    fig, ax = plt.subplots(1, 1, figsize=(15, 5))
    
    x = range(len(targets))
    ax.plot(x, targets, 'b-', label='Observed', linewidth=1.5, alpha=0.8)
    ax.plot(x, preds, 'r-', label='Predicted', linewidth=1.5, alpha=0.8)
    
    ax.set_xlabel('Time Step')
    ax.set_ylabel('Flow (Normalized)')
    ax.set_title(f'{task_name} - Predicted vs Observed Time Series')
    ax.legend()
    ax.grid(True, alpha=0.3)
    
    plt.tight_layout()
    plt.show()

# Plot if results exist
if 'task1_results' in dir() and task1_results is not None:
    plot_time_series(task1_results, "Task 1")

if 'task2_results' in dir() and task2_results is not None:
    plot_time_series(task2_results, "Task 2")

In [ ]:
# === 8.4 SUMMARY ===

print("="*60)
print(" PIPELINE SUMMARY")
print("="*60)

print(f"\\n📁 Configuration:")
print(f"   Dataset: {cfg.BASE_DIR}")
print(f"   Basins used: {cfg.NUM_BASINS}")
print(f"   Sequence length: {cfg.SEQ_LENGTH} days")
print(f"   Device: {device}")

print(f"\\n🔧 Hyperparameters:")
print(f"   Hidden dim: {cfg.HIDDEN_DIM}")
print(f"   Dropout: {cfg.DROPOUT}")
print(f"   Learning rate: {cfg.LEARNING_RATE}")
print(f"   Epochs: {cfg.EPOCHS}")
print(f"   Batch size: {cfg.BATCH_SIZE}")

print(f"\\n📊 Results:")
if 'task1_results' in dir() and task1_results is not None:
    print(f"   Task 1 (t+2) NSE: {task1_results['test_nse']:.4f}")
else:
    print("   Task 1: Not run")

if 'task2_results' in dir() and task2_results is not None:
    print(f"   Task 2 (t+1..t+5) NSE: {task2_results['test_nse']:.4f}")
else:
    print("   Task 2: Not run")

print(f"\\n📂 Results saved to: {cfg.RESULTS_DIR}/")
print("="*60)